<a href="https://colab.research.google.com/github/ianellisjones/usn/blob/main/USN_Carrier_%2B_Amphib_Tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
"""
US NAVY FLEET TRACKER (IEJ)

A scraping utility to track the deployment history of US Navy
Aircraft Carriers (CVN) and Amphibious Ships (LHA/LHD/LPD/LSD).
"""

import csv
import re
from pathlib import Path
from typing import List, Tuple, Dict

import requests
from bs4 import BeautifulSoup

# --- CONFIGURATION ---

FLEET_URLS: List[str] = [
    # Aircraft Carriers (CVN)
    "http://uscarriers.net/cvn68history.htm", # USS Nimitz
    "http://uscarriers.net/cvn69history.htm", # USS Eisenhower
    "http://uscarriers.net/cvn70history.htm", # USS Carl Vinson
    "http://uscarriers.net/cvn71history.htm", # USS Theodore Roosevelt
    "http://uscarriers.net/cvn72history.htm", # USS Abraham Lincoln
    "http://uscarriers.net/cvn73history.htm", # USS George Washington
    "http://uscarriers.net/cvn74history.htm", # USS John C. Stennis
    "http://uscarriers.net/cvn75history.htm", # USS Harry S. Truman
    "http://uscarriers.net/cvn76history.htm", # USS Ronald Reagan
    "http://uscarriers.net/cvn77history.htm", # USS George H.W. Bush
    "http://uscarriers.net/cvn78history.htm", # USS Gerald R. Ford

    # Amphibious Assault Ships (LHA/LHD)
    "http://uscarriers.net/lhd1history.htm", # USS Wasp
    "http://uscarriers.net/lhd2history.htm", # USS Essex
    "http://uscarriers.net/lhd3history.htm", # USS Kearsarge
    "http://uscarriers.net/lhd4history.htm", # USS Boxer
    "http://uscarriers.net/lhd5history.htm", # USS Bataan
    "http://uscarriers.net/lhd7history.htm", # USS Iwo Jima
    "http://uscarriers.net/lhd8history.htm", # USS Makin Island
    "http://uscarriers.net/lha6history.htm", # USS America
    "http://uscarriers.net/lha7history.htm", # USS Tripoli

    # Amphibious Transport Docks (LPD)
    "http://uscarriers.net/lpd17history.htm", # USS San Antonio
    "http://uscarriers.net/lpd18history.htm", # USS New Orleans
    "http://uscarriers.net/lpd19history.htm", # USS Mesa Verde
    "http://uscarriers.net/lpd20history.htm", # USS Green Bay
    "http://uscarriers.net/lpd21history.htm", # USS New York
    "http://uscarriers.net/lpd22history.htm", # USS San Diego
    "http://uscarriers.net/lpd23history.htm", # USS Anchorage
    "http://uscarriers.net/lpd24history.htm", # USS Arlington
    "http://uscarriers.net/lpd25history.htm", # USS Somerset
    "http://uscarriers.net/lpd26history.htm", # USS John P. Murtha
    "http://uscarriers.net/lpd27history.htm", # USS Portland
    "http://uscarriers.net/lpd28history.htm", # USS Fort Lauderdale
    "http://uscarriers.net/lpd29history.htm", # USS Richard M. McCool Jr.

    # Dock Landing Ships (LSD)
    "http://uscarriers.net/lsd44history.htm", # USS Gunston Hall
    "http://uscarriers.net/lsd46history.htm", # USS Tortuga
    "http://uscarriers.net/lsd50history.htm", # USS Carter Hall
    "http://uscarriers.net/lsd51history.htm", # USS Oak Hill
    "http://uscarriers.net/lsd42history.htm", # USS Germantown
    "http://uscarriers.net/lsd45history.htm", # USS Comstock
    "http://uscarriers.net/lsd48history.htm", # USS Ashland
    "http://uscarriers.net/lsd49history.htm", # USS Harpers Ferry
    "http://uscarriers.net/lsd52history.htm", # USS Pearl Harbor
    "http://uscarriers.net/lsd47history.htm", # USS Rushmore
]

OUTPUT_FILENAME = "big_deck_status.csv"
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'


def fetch_history_text(url: str, char_limit: int = 50000) -> str:
    try:
        response = requests.get(url, headers={'User-Agent': USER_AGENT}, timeout=20)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        full_text = soup.get_text(separator='\n')
        lines = [line.strip() for line in full_text.split('\n') if line.strip()]
        clean_text = '\n'.join(lines)
        return clean_text[-char_limit:] if len(clean_text) > char_limit else clean_text
    except requests.RequestException as e:
        return f"ERROR: {str(e)}"


def parse_status_entry(text_block: str) -> Tuple[str, str]:
    lines = text_block.split('\n')

    current_year = "Unknown"
    years_found = re.findall(r'(202[3-6])', text_block)

    if years_found:
        priority_years = [y for y in years_found if y in ['2024', '2025']]
        current_year = priority_years[-1] if priority_years else years_found[-1]

    processed_lines = []
    running_year = current_year

    for line in lines:
        year_match = re.search(r'^202[3-6]', line)
        if year_match:
            running_year = year_match.group(0)
        processed_lines.append({'text': line, 'year': running_year})

    keywords = [
        "moored", "anchored", "underway", "arrived", "departed",
        "transited", "operations", "returned", "participated", "conducted",
        "moved to", "visited", "pulled into", "sea trials", "flight deck certification",
        "undocked"
    ]

    allowed_years = ["2024", "2025", "2026"]

    for entry in reversed(processed_lines):
        text_lower = entry['text'].lower()
        year = entry['year']

        if year in allowed_years and any(k in text_lower for k in keywords):
            if text_lower.strip().startswith("from ") and " - " in text_lower:
                continue
            return year, entry['text']

    return current_year, "No status found."


def categorize_location(text: str) -> str:
    """
    Maps status text to standardized high-level location tags.
    Logic: Sorts matches by (End Index DESC, Length DESC).
    This ensures "Departed San Diego" wins over "San Diego" because it is longer
    and ends at the same position.
    """
    text = text.lower()

    location_map = {
        # Movement Overrides (Mapped to Oceans)
        "Pacific Ocean": ["departed san diego", "departed pearl harbor", "pacific ocean", "pacific"],
        "Atlantic Ocean": ["departed norfolk", "departed mayport", "atlantic ocean", "atlantic", "departed little creek", "off the coast of panama city"],

        # Ports
        "San Diego": ["san diego", "north island", "camp pendleton", "nassco", "bae systems", "point loma", "a-175", "a-173"],
        "Norfolk": ["norfolk", "portsmouth", "virginia beach", "newport news", "little creek", "berth 6, pier 2"], # Added new mapping
        "Bremerton": ["bremerton", "kitsap"],
        "Yokosuka": ["yokosuka"],
        "Pearl Harbor": ["pearl harbor"],
        "Mayport": ["mayport"],
        "Everett": ["everett"],
        "Singapore": ["singapore", "changi"],
        "Bahrain": ["bahrain", "manama"],
        "Dubai": ["dubai", "jebel ali"],
        "Busan": ["busan"],
        "Guam": ["guam", "apra"],
        "Sasebo": ["sasebo", "juliet basin"],
        "Manila": ["manila", "subic", "philippines"],
        "Klang": ["malaysia", "klang"],
        "Okinawa": ["okinawa"],
        "Pascagoula": ["pascagoula", "ingalls"],
        "Portland, OR": ["swan island", "portland"],

        # Regions
        "South China Sea": ["south china sea", "spratly", "luzon"],
        "Western Pacific": ["western pacific", "westpac", "san bernardino", "philippine sea"],
        "Persian Gulf": ["persian gulf", "arabian gulf"],
        "Red Sea": ["red sea", "bab el-mandeb"],
        "Gulf of Oman": ["gulf of oman"],
        "Gulf of Aden": ["gulf of aden"],
        "Mediterranean": ["mediterranean", "adriatic", "gibraltar"],
        "Caribbean Sea": ["caribbean", "st. croix", "trinidad", "tobago", "puerto rico", "virgin islands", "st. thomas"],
        "North Sea": ["north sea", "norwegian sea", "oslo"],
        "Suez Canal": ["suez"],
        "Panama Canal": ["panama canal"],

        # Oceans
        "Indian Ocean": ["indian ocean"],
    }

    matches = []

    for label, keywords in location_map.items():
        for k in keywords:
            # rfind gets the last occurrence start index
            start_index = text.rfind(k)
            if start_index != -1:
                end_index = start_index + len(k)
                # Store tuple: (End Index, Length, Label)
                matches.append((end_index, len(k), label))

    if matches:
        # Sort by End Index DESC, then Length DESC
        # This means if two matches end at the same spot (San Diego vs Departed San Diego),
        # the longer one (Departed...) comes first.
        matches.sort(key=lambda x: (x[0], x[1]), reverse=True)
        return matches[0][2]

    return "Underway / Unknown"


def extract_date(text: str) -> str:
    pattern = r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\.?\s+\d{1,2}'
    matches = re.findall(pattern, text, re.IGNORECASE)
    return matches[-1] if matches else "Date Unspecified"


def main():
    print(f"{'='*90}")
    print(f"US NAVY BIG DECK TRACKER (CVN + LHA/LHD/LPD/LSD)")
    print(f"{'='*90}\n")

    results: List[Dict[str, str]] = []

    for url in FLEET_URLS:
        # Updated Regex to include LSD
        hull_match = re.search(r'((?:cvn|lha|lhd|lpd|lsd)\d+)', url, re.IGNORECASE)
        hull = hull_match.group(1).upper() if hull_match else "UNK"

        raw_text = fetch_history_text(url)

        if "ERROR" in raw_text:
            year, status, loc_tag, date_str = "Error", raw_text, "Error", "Error"
        else:
            year, status = parse_status_entry(raw_text)
            loc_tag = categorize_location(status)
            date_str = extract_date(status)

            if date_str == "Date Unspecified":
                date_str = year

        results.append({
            "Hull": hull,
            "Location": loc_tag,
            "Date": date_str,
            "Status Sentence": status,
            "Source URL": url
        })

        print(f"[{hull}] [{loc_tag}] [{date_str}] {status}")

    try:
        output_path = Path(OUTPUT_FILENAME)
        with output_path.open(mode='w', newline='', encoding='utf-8') as csvfile:
            fieldnames = ["Hull", "Location", "Date", "Status Sentence", "Source URL"]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(results)

        print(f"\n{'='*90}")
        print(f"SUCCESS: Report saved to '{output_path.absolute()}'")
        print(f"{'='*90}")

    except PermissionError:
        print(f"\nERROR: Could not write to {OUTPUT_FILENAME}. Is the file open?")

if __name__ == "__main__":
    main()

US NAVY BIG DECK TRACKER (CVN + LHA/LHD/LPD/LSD)

[CVN68] [Western Pacific] [Nov. 18] From November 8-11, the Nimitz CSG conducted operations  off the coast of Brunei; Conducted operations northeast of Spratly Islands from Nov. 12-13; Conducted operations off the southwest coast of Luzon from Nov. 14-17; Transited the San Bernardino Strait northbound on Nov. 18.
[CVN69] [Norfolk] [Jan. 8] September 26, USS Dwight D. Eisenhower moored at Pier 12N on Naval Station Norfolk after a six-day underway for TRACOM-CQ, in the Jacksonville Op. Area; Moved "dead-stick" to Super Pier 5N in Norfolk Naval Shipyard, for a Planned Incremental Availability (PIA), on Jan. 8.
[CVN70] [San Diego] [September 16] September 16, The Carl Vinson moored at Juliet Pier on Naval Air Station North Island after a three-day underway for ammo offload.
[CVN71] [San Diego] [Nov. 10] November  8, USS Theodore Roosevelt moored at Berth Lima on Naval Air Station North Island   after a 12-day underway for Tailored Ship's Tr

In [5]:
"""
US NAVY FLEET MAPPER (v8.0 - Complete Fleet)

Visualizes the output of the 'Big Deck Tracker' (big_deck_status.csv).
Covers: CVN, LHA, LHD, LPD, and LSD.
"""

import pandas as pd
import folium
import os
from folium.plugins import MarkerCluster, BeautifyIcon

# --- CONFIGURATION ---
INPUT_FILE = "big_deck_status.csv"
OUTPUT_FILE = "fleet_map.html"

# --- COORDINATE DICTIONARY ---
# Maps the "Location" column from the CSV to [Lat, Lon]
LOCATION_COORDS = {
    # US Ports / Shipyards
    "norfolk": [36.96, -76.30],
    "newport news": [36.96, -76.30], # Consolidated to Norfolk pin
    "portsmouth": [36.96, -76.30],   # Consolidated to Norfolk pin
    "virginia beach": [36.85, -75.97],

    "san diego": [32.68, -117.18],
    "north island": [32.70, -117.20],
    "camp pendleton": [33.29, -117.40],
    "bremerton": [47.55, -122.64],
    "kitsap": [47.70, -122.70],
    "everett": [47.98, -122.22],
    "mayport": [30.39, -81.42],
    "pearl harbor": [21.35, -157.97],
    "pascagoula": [30.34, -88.56],
    "ingalls": [30.34, -88.56],
    "groton": [41.39, -72.08],
    "portland, or": [45.56, -122.71],

    # Foreign Ports
    "yokosuka": [35.29, 139.66],
    "sasebo": [33.16, 129.71],
    "busan": [35.10, 129.11],
    "guam": [13.44, 144.65],
    "singapore": [1.30, 103.85],
    "changi": [1.35, 104.00],
    "bahrain": [26.22, 50.61],
    "dubai": [25.26, 55.30],
    "jebel ali": [25.00, 55.06],
    "manila": [14.59, 120.97],
    "subic": [14.80, 120.27],
    "okinawa": [26.30, 127.80],
    "klang": [3.00, 101.38],
    "trinidad": [10.66, -61.51],
    "tobago": [11.25, -60.67],
    "st. croix": [17.71, -64.88],
    "st. thomas": [18.33, -64.92],
    "virgin islands": [18.33, -64.92],
    "rostock": [54.15, 12.10],
    "plymouth": [50.37, -4.14],
    "oslo": [59.91, 10.75],
    "split": [43.50, 16.44],

    # Regions / Seas
    "south china sea": [12.00, 114.00],
    "spratly": [10.00, 114.00],
    "luzon": [16.00, 121.00],
    "philippine sea": [20.00, 130.00],
    "western pacific": [15.00, 135.00], # WestPac
    "westpac": [15.00, 135.00],
    "san bernardino": [12.53, 124.28],
    "persian gulf": [27.00, 51.00],
    "arabian gulf": [27.00, 51.00],
    "red sea": [20.00, 38.00],
    "gulf of oman": [24.00, 58.00],
    "gulf of aden": [12.00, 48.00],
    "mediterranean": [35.00, 18.00],
    "adriatic": [42.00, 15.00],
    "caribbean sea": [15.00, -75.00],
    "puerto rico": [18.22, -66.59],
    "north sea": [56.00, 3.00],
    "norwegian sea": [66.00, 5.00],
    "strait of gibraltar": [35.95, -5.60],
    "suez canal": [30.60, 32.33],
    "panama canal": [9.10, -79.67],
    "bab el-mandeb": [12.58, 43.33],

    # Oceans
    "atlantic ocean": [33.00, -60.00],
    "pacific ocean": [25.00, -150.00],
    "indian ocean": [-5.00, 80.00],

    # Fallback
    "underway / unknown": [0.0, 0.0]
}

def get_location_key_and_coords(text):
    """Matches scraper text to coordinate keys."""
    if not isinstance(text, str): return "unknown", LOCATION_COORDS["underway / unknown"]
    text = text.lower()

    # Try exact match first (fast)
    if text in LOCATION_COORDS:
        return text, LOCATION_COORDS[text]

    # Try partial match (slower but robust)
    # Sort keys by length to match specific locations ("Newport News") before general ones ("Newport")
    sorted_keys = sorted(LOCATION_COORDS.keys(), key=len, reverse=True)
    for location in sorted_keys:
        if location in text:
            return location, LOCATION_COORDS[location]

    return "unknown", LOCATION_COORDS["underway / unknown"]

def generate_map():
    print(f"{'='*60}")
    print(f"OFFICIAL FLEET MAPPER (CVN + LHA/LHD/LPD/LSD)")
    print(f"{'='*60}")

    if not os.path.exists(INPUT_FILE):
        print(f"ERROR: {INPUT_FILE} not found. Run big_deck_scraper.py first.")
        return

    # Locations Database for Aggregation
    locations_db = {}

    def add_to_db(loc_key, coords, ship_data):
        # Consolidate visual pins for Hampton Roads
        if loc_key in ["newport news", "portsmouth", "virginia beach"]:
            loc_key = "norfolk"

        if loc_key not in locations_db:
            locations_db[loc_key] = {"coords": coords, "ships": []}
        locations_db[loc_key]["ships"].append(ship_data)

    print(f"Reading fleet data from {INPUT_FILE}...")
    try:
        df = pd.read_csv(INPUT_FILE)
    except Exception as e:
        print(f"Error reading CSV: {e}")
        return

    mapped_count = 0

    for _, row in df.iterrows():
        loc_tag = str(row['Location'])
        loc_key, coords = get_location_key_and_coords(loc_tag)

        hull = row['Hull']
        # Determine Ship Type for coloring
        if "CVN" in hull:
            s_type = "CVN"
        else:
            s_type = "AMPHIB" # Covers LHA, LHD, LPD, LSD

        add_to_db(loc_key, coords, {
            "hull": hull,
            "date": row['Date'],
            "status": row['Status Sentence'],
            "type": s_type
        })
        mapped_count += 1

    # Render Map
    m = folium.Map(location=[25, 0], zoom_start=3, tiles="CartoDB positron")

    for loc_key, data in locations_db.items():
        coords = data['coords']
        ships = data['ships']

        # Icon Logic: Blue if ANY Carrier present, Red if only Amphibs
        has_cvn = any(s['type'] == "CVN" for s in ships)
        color = "blue" if has_cvn else "red"
        icon = "plane" if has_cvn else "anchor"

        # Special styling for Null Island (Unknowns)
        if coords == [0.0, 0.0]:
            color = "gray"
            icon = "question"
            loc_label = "LOCATION UNKNOWN / TRANSIT"
        else:
            loc_label = loc_key.title()

        hull_list = [s['hull'] for s in ships]
        tooltip_text = f"{loc_label}: {', '.join(hull_list)}"

        # Build Popup Table
        popup_html = f"""
        <div style="font-family:sans-serif; width:350px; overflow-y:auto; max-height:300px;">
            <h4 style="margin-bottom:5px; border-bottom:2px solid {color};">{loc_label.upper()}</h4>
            <table style="width:100%; font-size:11px; border-collapse:collapse;">
        """

        # Sort ships: CVNs first, then others
        ships.sort(key=lambda x: x['type'] != "CVN")

        for s in ships:
            badge_color = "darkblue" if s['type'] == "CVN" else "darkred"
            popup_html += f"""
            <tr style="border-bottom:1px solid #eee;">
                <td style="padding:5px; vertical-align:top; width:70px;">
                    <b style="color:{badge_color};">{s['hull']}</b>
                </td>
                <td style="padding:5px;">
                    <span style="color:gray; font-size:9px;">{s['date']}</span><br>
                    {s['status'][:200]}
                </td>
            </tr>
            """

        popup_html += "</table></div>"

        folium.Marker(
            location=coords,
            popup=folium.Popup(popup_html, max_width=370),
            tooltip=tooltip_text,
            icon=folium.Icon(color=color, icon=icon, prefix='fa')
        ).add_to(m)

    # Legend
    title_html = '''
             <div style="position: fixed;
                         top: 10px; right: 10px; width: 190px; height: 110px;
                         z-index:9999; font-size:12px;
                         background-color: white; opacity: 0.9;
                         padding: 10px; border-radius: 5px; border: 1px solid grey;">
                 <b>US Navy Fleet Overview</b><br>
                 <hr style="margin:5px 0;">
                 <i class="fa fa-plane" style="color:blue; margin-right:5px;"></i> Carrier Strike Group<br>
                 <i class="fa fa-anchor" style="color:red; margin-right:5px;"></i> Amphibious Group<br>
                 <i class="fa fa-question" style="color:gray; margin-right:5px;"></i> Unknown/Transit
             </div>
             '''
    m.get_root().html.add_child(folium.Element(title_html))

    m.save(OUTPUT_FILE)
    print(f"\nSUCCESS: Map generated at '{os.path.abspath(OUTPUT_FILE)}'")
    print(f"Processed {mapped_count} ships into {len(locations_db)} locations.")

if __name__ == "__main__":
    generate_map()

OFFICIAL FLEET MAPPER (CVN + LHA/LHD/LPD/LSD)
Reading fleet data from big_deck_status.csv...

SUCCESS: Map generated at '/content/fleet_map.html'
Processed 43 ships into 8 locations.
